<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=3bc97c3943056107f89e9ff4c75eeb0c20e6c4ea708559dee1cae4f5bad6b62e
  Stored in directory: /home/runner/.cache/pip/wheels/ec/b6/81/d781ff040ee9432384560f29596ee14ca79d70529d4228e50f
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.33.1
    Uninstalling protobuf-6.33.1:
      Successfully uninstalled protobuf-6.33.1


  Attempting uninstall: yfinance
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

    Found existing installation: yfinance 0.2.66
    Uninstalling yfinance-0.2.66:
      Successfully uninstalled yfinance-0.2.66
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-12-05 09:15:52
-------------------
qualified stocks: 86
with latest results: 25
still star stocks: 15
-------------------
Initial Investment:  1.28 C
CY Investment:  1.55 C
Reserve:  4.95 L
Current:  1.42 C
-------------------
Today PnL: -2.31 L (-1.61%)
Current PnL: -26.60 L (-17.18%)
CY Booked + Current PnL: -12.31 L (-7.95%)
-------------------
Total profit:  1.24 L
Total loss:  -27.85 L
-------------------
Total Booked + Current PnL: 14.67 L (11.47%)
Total Booked PnL: 41.27 L (32.29%)
Curr Year Booked PnL: 14.29 L (10.09%)
Prev Year Booked PnL: 26.98 L (21.11%)
Est FTT:  2.33 C
Est FTT PnL: 91.23 L (64.4%)
Deployed:  1.28 C
Current:  1.42 C
CAGR/XIRR %: 5.83%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT','Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'FTT Amt', 'Current P/L', 'Current', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%',  'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
39,INDIGOPNTS,1408.00,1.59,-7.23,7.81,0.02,12648.0,-12615.0,161944.0,147.21,69.0,M-SC,15.60,234.0,-1.00,1.18,40.86,OX40N,NTT,PAINTS
77,TTKPRESTIG,770.00,-0.70,-17.29,21.02,0.09,17520.0,-17428.0,83349.0,93.38,35.0,M-SC,2.72,253.0,-0.99,0.61,9.50,OX40N,NTT,DURABLES
43,ITC,452.00,-0.15,-1.67,12.31,10.44,24226.0,-3340.0,196798.0,-41.15,44.0,X-LC,1.59,5.0,-0.14,1.44,3.80,X40,NTT,FMCG
35,ICICIGI,2252.93,0.50,5.75,13.66,20.19,25182.0,10025.0,184345.0,-17.20,47.0,X-MC,6.62,68.0,0.40,1.35,21.73,X40,ATH,INSURANCE
50,MASFIN,398.61,-2.75,-4.70,28.07,22.05,26210.0,-4605.0,93375.0,-17.80,50.0,H-SC,6.86,164.0,-0.18,0.68,35.73,XR,ATH,FINANCE


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
80,VALIANTORG,838.00,8.44,-42.46,202.64,74.15,190777.0,-69459.0,94146.0,-269.06,49.0,H-SC,19.87,149.0,-0.36,0.69,20.68,XR,NTT,CHEMICALS
29,HAPPSTMNDS,1480.71,3.06,-36.04,189.00,84.85,154935.0,-46189.0,81976.0,-23.94,57.0,H-SC,13.99,147.0,-0.30,0.60,5.21,AR,ATH,IT
40,INDUSINDBK,1800.00,2.07,-36.18,108.24,32.89,53331.0,-27936.0,49271.0,-708.79,65.0,L-MC,6.22,259.0,-0.52,0.36,35.69,XR,NTT,BANKS
73,TCS,4389.97,1.63,-12.19,35.84,19.28,108878.0,-42167.0,303789.0,-23.30,69.0,X-LC,6.53,1.0,-0.39,2.22,12.31,X40,ATH,IT
39,INDIGOPNTS,1408.00,1.59,-7.23,7.81,0.02,12648.0,-12615.0,161944.0,147.21,69.0,M-SC,15.60,234.0,-1.00,1.18,40.86,OX40N,NTT,PAINTS


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
15,CAMS,4762.00,-79.54,-79.30,502.94,24.79,262168.0,-199729.0,52127.0,-80.20,7.0,X-SC,1.57,86.0,-0.76,0.38,0.00,X40N,NTT,MISC
44,JCHAC,2282.00,-4.05,-34.16,51.86,-0.01,43641.0,-43654.0,84151.0,17117.39,8.0,M-SC,0.99,233.0,-1.00,0.62,0.00,OX40N,NTT,AC
69,SURYODAY,216.00,-2.78,-21.97,57.68,23.03,80596.0,-39341.0,139730.0,51.32,42.0,H-SC,10.03,167.0,-0.49,1.02,38.64,XR,NTT,BANKS
50,MASFIN,398.61,-2.75,-4.70,28.07,22.05,26210.0,-4605.0,93375.0,-17.80,50.0,H-SC,6.86,164.0,-0.18,0.68,35.73,XR,ATH,FINANCE
22,DIXON,18931.72,-2.69,-12.39,38.79,21.58,47622.0,-17370.0,122769.0,-55.87,25.0,X-MC,5.76,56.0,-0.36,0.90,8.69,X40N,ATH,IT


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
37,IEX,219.0,-0.79,0.24,48.26,48.63,96733.0,488.0,200442.0,-33.4,63.0,H-SC,14.74,136.0,0.01,1.47,11.83,XR,NTT,MISC


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
25,FINCABLES,1641.55,-0.42,-6.44,122.22,107.91,168831.0,-9507.0,138137.0,-23.79,31.0,M-SC,11.22,216.0,-0.06,1.01,0.00,OX40N,ATH,CABLES
39,INDIGOPNTS,1408.00,1.59,-7.23,7.81,0.02,12648.0,-12615.0,161944.0,147.21,69.0,M-SC,15.60,234.0,-1.00,1.18,40.86,OX40N,NTT,PAINTS
77,TTKPRESTIG,770.00,-0.70,-17.29,21.02,0.09,17520.0,-17428.0,83349.0,93.38,35.0,M-SC,2.72,253.0,-0.99,0.61,9.50,OX40N,NTT,DURABLES
47,KANSAINER,340.00,-1.67,-23.64,48.61,13.47,100093.0,-63756.0,205911.0,-68.70,32.0,H-SC,2.38,158.0,-0.64,1.51,4.70,XY24,NTT,PAINTS
66,SIS,528.00,0.14,-24.36,60.32,21.26,50659.0,-27048.0,83984.0,1980.83,47.0,H-SC,4.05,166.0,-0.53,0.61,13.47,OX40N,NTT,MISC


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
84,WIPRO,420.0,0.83,6.26,62.12,72.27,111850.0,10606.0,180054.0,-8.8,72.0,M-LC,2.95,99.0,0.09,1.32,12.82,XR,NTT,IT
37,IEX,219.0,-0.79,0.24,48.26,48.63,96733.0,488.0,200442.0,-33.4,63.0,H-SC,14.74,136.0,0.01,1.47,11.83,XR,NTT,MISC


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
84,WIPRO,420.00,0.83,6.26,62.12,72.27,111850.0,10606.0,180054.0,-8.80,72.0,M-LC,2.95,99.0,0.09,1.32,12.82,XR,NTT,IT
37,IEX,219.00,-0.79,0.24,48.26,48.63,96733.0,488.0,200442.0,-33.40,63.0,H-SC,14.74,136.0,0.01,1.47,11.83,XR,NTT,MISC
38,INDIAMART,4810.62,-1.89,-1.79,110.48,106.72,133827.0,-2204.0,121132.0,-51.39,35.0,H-SC,1.54,139.0,-0.02,0.89,21.17,AR,ATH,MISC
85,ZYDUSLIFE,1286.17,-0.09,-1.50,37.69,35.63,77806.0,-3138.0,206436.0,-15.41,41.0,H-MC,3.42,119.0,-0.04,1.51,14.93,AR,ATH,PHARMA
25,FINCABLES,1641.55,-0.42,-6.44,122.22,107.91,168831.0,-9507.0,138137.0,-23.79,31.0,M-SC,11.22,216.0,-0.06,1.01,0.00,OX40N,ATH,CABLES


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
15,CAMS,4762.00,-79.54,-79.30,502.94,24.79,262168.0,-199729.0,52127.0,-80.20,7.0,X-SC,1.57,86.0,-0.76,0.38,0.00,X40N,NTT,MISC
9,BAJAJHFL,181.50,-0.70,-21.24,88.49,48.45,139740.0,-42590.0,157916.0,-26.96,12.0,X-MC,9.42,64.0,-0.30,1.16,0.00,X40N,ATH,FINANCE
11,BATAINDIA,2096.00,-2.14,-38.30,119.13,35.20,94577.0,-49280.0,79390.0,4.12,22.0,X-SC,14.45,92.0,-0.52,0.58,0.00,X40,NTT,FOOTWEAR
22,DIXON,18931.72,-2.69,-12.39,38.79,21.58,47622.0,-17370.0,122769.0,-55.87,25.0,X-MC,5.76,56.0,-0.36,0.90,8.69,X40N,ATH,IT
23,DMART,5391.45,-0.03,-7.12,37.94,28.12,74146.0,-14977.0,195430.0,-19.86,29.0,X-LC,4.21,19.0,-0.20,1.43,14.83,X40N,ATH,FMCG


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
3,ACC,3906.0,-1.04,-23.47,114.50,64.15,208504.0,-55851.0,182100.0,-55.44,42.0,X-SC,1.48,82.0,-0.27,1.33,2.15,XY24,BTT,CEMENT
8,AWL,485.0,0.75,-21.07,94.47,53.50,225006.0,-63574.0,238177.0,-61.84,31.0,X-MC,1.55,58.0,-0.28,1.74,5.79,XY24,NTT,FMCG
15,CAMS,4762.0,-79.54,-79.30,502.94,24.79,262168.0,-199729.0,52127.0,-80.20,7.0,X-SC,1.57,86.0,-0.76,0.38,0.00,X40N,NTT,MISC
43,ITC,452.0,-0.15,-1.67,12.31,10.44,24226.0,-3340.0,196798.0,-41.15,44.0,X-LC,1.59,5.0,-0.14,1.44,3.80,X40,NTT,FMCG
36,ICICIPRULI,790.0,0.53,1.87,28.56,30.97,51422.0,3310.0,180048.0,-22.06,50.0,X-MC,2.22,75.0,0.06,1.32,14.67,X40,ATH,INSURANCE


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
11,BATAINDIA,2096.00,-2.14,-38.30,119.13,35.20,94577.0,-49280.0,79390.0,4.12,22.0,X-SC,14.45,92.0,-0.52,0.58,0.00,X40,NTT,FOOTWEAR
15,CAMS,4762.00,-79.54,-79.30,502.94,24.79,262168.0,-199729.0,52127.0,-80.20,7.0,X-SC,1.57,86.0,-0.76,0.38,0.00,X40N,NTT,MISC
9,BAJAJHFL,181.50,-0.70,-21.24,88.49,48.45,139740.0,-42590.0,157916.0,-26.96,12.0,X-MC,9.42,64.0,-0.30,1.16,0.00,X40N,ATH,FINANCE
55,QUESS,424.00,-1.41,-31.10,107.33,42.84,48069.0,-20220.0,44786.0,-54.82,35.0,X-SC,37.33,83.0,-0.42,0.33,0.00,XY24,NTT,MISC
54,PGHH,17907.65,0.25,-5.36,41.33,33.76,78554.0,-10755.0,190065.0,-33.18,32.0,X-MC,4.67,57.0,-0.14,1.39,0.25,X40,ATH,FMCG


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
55,QUESS,424.00,-1.41,-31.10,107.33,42.84,48069.0,-20220.0,44786.0,-54.82,35.0,X-SC,37.33,83.0,-0.42,0.33,0.00,XY24,NTT,MISC
15,CAMS,4762.00,-79.54,-79.30,502.94,24.79,262168.0,-199729.0,52127.0,-80.20,7.0,X-SC,1.57,86.0,-0.76,0.38,0.00,X40N,NTT,MISC
59,RELAXO,1176.00,0.44,-47.17,188.27,52.31,144394.0,-68465.0,76695.0,-43.76,38.0,X-SC,5.34,91.0,-0.47,0.56,2.66,X40N,NTT,FOOTWEAR
11,BATAINDIA,2096.00,-2.14,-38.30,119.13,35.20,94577.0,-49280.0,79390.0,4.12,22.0,X-SC,14.45,92.0,-0.52,0.58,0.00,X40,NTT,FOOTWEAR
22,DIXON,18931.72,-2.69,-12.39,38.79,21.58,47622.0,-17370.0,122769.0,-55.87,25.0,X-MC,5.76,56.0,-0.36,0.90,8.69,X40N,ATH,IT


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
73,TCS,4389.97,1.63,-12.19,35.84,19.28,108878.0,-42167.0,303789.0,-23.30,69.0,X-LC,6.53,1.0,-0.39,2.22,12.31,X40,ATH,IT
41,INFY,2275.00,0.75,9.57,41.35,54.87,141758.0,29936.0,342824.0,-14.57,69.0,X-LC,3.24,2.0,0.21,2.51,18.64,X40,BTT,IT
75,TMPV,600.00,-0.14,-15.65,68.56,42.18,159846.0,-43263.0,233147.0,-24.30,31.0,X-LC,3.94,3.0,-0.27,1.71,0.99,XY24,NTT,AUTO
81,VBL,671.64,-0.82,-4.33,41.82,35.67,126362.0,-13685.0,302157.0,-15.77,55.0,X-LC,5.33,4.0,-0.11,2.21,8.92,X40N,ATH,FMCG
43,ITC,452.00,-0.15,-1.67,12.31,10.44,24226.0,-3340.0,196798.0,-41.15,44.0,X-LC,1.59,5.0,-0.14,1.44,3.80,X40,NTT,FMCG


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
40,INDUSINDBK,1800.00,2.07,-36.18,108.24,32.89,53331.0,-27936.0,49271.0,-708.79,65.0,L-MC,6.22,259.0,-0.52,0.36,35.69,XR,NTT,BANKS
6,ASIANTILES,137.00,-1.92,-16.71,118.12,81.67,92294.0,-15674.0,78136.0,7022.22,39.0,L-SC,16.30,271.0,-0.17,0.57,51.31,XR,NTT,CERAMICS
50,MASFIN,398.61,-2.75,-4.70,28.07,22.05,26210.0,-4605.0,93375.0,-17.80,50.0,H-SC,6.86,164.0,-0.18,0.68,35.73,XR,ATH,FINANCE
13,BSOFT,831.70,-0.21,-21.14,91.20,50.79,101164.0,-29728.0,110925.0,1.28,74.0,H-SC,5.20,151.0,-0.29,0.81,29.15,XR,ATH,IT
39,INDIGOPNTS,1408.00,1.59,-7.23,7.81,0.02,12648.0,-12615.0,161944.0,147.21,69.0,M-SC,15.60,234.0,-1.00,1.18,40.86,OX40N,NTT,PAINTS


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
39,INDIGOPNTS,1408.00,1.59,-7.23,7.81,0.02,12648.0,-12615.0,161944.0,147.21,69.0,M-SC,15.60,234.0,-1.00,1.18,40.86,OX40N,NTT,PAINTS
5,ASIANPAINT,3460.25,-0.16,3.20,17.19,20.95,44665.0,8061.0,259829.0,2.18,78.0,X-LC,18.45,31.0,0.18,1.90,39.98,X40,ATH,PAINTS
13,BSOFT,831.70,-0.21,-21.14,91.20,50.79,101164.0,-29728.0,110925.0,1.28,74.0,H-SC,5.20,151.0,-0.29,0.81,29.15,XR,ATH,IT
41,INFY,2275.00,0.75,9.57,41.35,54.87,141758.0,29936.0,342824.0,-14.57,69.0,X-LC,3.24,2.0,0.21,2.51,18.64,X40,BTT,IT
40,INDUSINDBK,1800.00,2.07,-36.18,108.24,32.89,53331.0,-27936.0,49271.0,-708.79,65.0,L-MC,6.22,259.0,-0.52,0.36,35.69,XR,NTT,BANKS


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,20.66
1,25,45.57
2,50,76.55


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    45.34
MC    29.97
LC    24.72
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     28.42
X40      22.27
X40N     10.79
XR       10.08
AR        9.44
XY25      9.38
OX40N     7.81
SR        1.04
MH        0.80
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    24.70
X-MC    23.25
X-LC    21.14
M-SC    12.27
X-SC     6.97
H-MC     4.94
M-MC     1.42
L-SC     1.40
M-LC     1.32
H-LC     1.23
L-LC     1.03
L-MC     0.36
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,17.34,-6.17,40.50
IT,12.76,-15.54,78.01
FINANCE,9.59,-18.22,68.18
ELECTRICAL,6.27,-9.54,49.67
PAINTS,6.23,-8.31,24.40
MISC,6.19,-53.72,116.69
INSURANCE,4.52,-2.49,38.61
PHARMA,4.03,-1.40,33.86
AUTO,3.47,-21.28,73.28


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3152809.0,21
AR,1284504.0,10
XR,1283987.0,13
X40,993270.0,14
X40N,975122.0,9
OX40N,722187.0,10
XY25,359776.0,6
SR,277531.0,2
MH,74251.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3602462.0,25
M-SC,1382276.0,15
X-MC,1305421.0,16
X-SC,965684.0,8
X-LC,873108.0,11
H-MC,403690.0,3
L-SC,269117.0,3
M-LC,111850.0,1
H-LC,65801.0,1


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1263440.0      6
           AR         884987.0      5
M-SC       XY24       788699.0      6
H-SC       XR         768453.0      7
X-SC       X40N       500587.0      3
X-MC       X40        454688.0      7
           XY24       386439.0      3
X-LC       X40        361216.0      5
H-SC       OX40N      333800.0      4
M-SC       OX40N      300938.0      5
X-SC       XY24       287731.0      3
H-SC       SR         277531.0      2
X-MC       X40N       274027.0      4
X-LC       XY24       232572.0      2
H-MC       AR         209762.0      2
X-LC       X40N       200508.0      2
H-MC       XY24       193928.0      1
X-MC       XY25       190267.0      2
L-SC       XR         181668.0      2
X-SC       X40        177366.0      2
M-SC       XR         168685.0      2
           AR         123954.0      2
M-LC       XR         111850.0      1
L-SC       OX40N       87449.0      1
X-LC       XY25        78812.0      2
H-SC       MH          74251.0      1
H-LC       AR          65801.0      1
L-MC       XR          53331.0      1
M-MC       XY25        51554.0      1
L-LC       XY25        39143.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 34.0 seconds
